# Second-order Optimization

To overcome the issues of vanilla gradient descent we introduce Newton's method, a second-order optimization scheme that exploits second-order derivatives.

# Optimality Conditions

Given an unconstrained optimization problem:
$$
\min_x f(x)
$$

A local minimizer of $f(x)$ must satisfy the following condition:
$$
\nabla f(x) = 0
$$
This condition is **necessary**.

Moreover, a **sufficient** (but not necessary) condition for a local minimizer is:
$$
\nabla^2 f(x) > 0
$$
which means that the Hessian $\nabla^2 f(x)$ is positive definite (i.e. all its eigenvalues are positive).

# Newton's Method

The necessary optimality conditions define a nonlinear system of equations:
$$
r(x) \triangleq \nabla f(x) = 0
$$
We can try to solve this system of equations with Newton's method, an iterative algorithm that produces successively better approximations to the zeroes of a function.

Newton's method approximates the function $r(x)$ in the neighborhood of the current guess $\bar{x}$ using a first-order Taylor's series expansion:
$$
r(\bar{x} + \Delta x) \approx r(\bar{x}) + \nabla r(\bar{x})^T \, \Delta x = 0
$$

We can solve this linear system of equations by computing the so-called "Newton's step":
$$
\Delta x = - \nabla r(\bar{x})^{-T} \, r(\bar{x})
$$

Finally, we update our guess by applying Newton's step:
$$
\bar{x} := \bar{x} + \Delta x
$$

We repeat this process until $||r(x)|| \le \text{threshold}$. Let's see an implementation of Newton's method for solving the equation $x^4 - x = 0$.

In [5]:
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interact, IntSlider

# the system of equations to solve is defined as: r(x)=0
def r(x):
    return x**4 -x

# function computing the gradient of r(x)
def grad_r(x):
    return 4*(x**3) - 1

In [6]:
def newtons_method_root_finding(n_iters):
    # n_iters: number of iterations to perform
    x_bar = -2.0 # initial guess
    
    # plot the function r(x)
    fig = plt.figure(figsize=(12,7))
    X = np.linspace(-2.1, 0.1, 100)
    plt.plot(X, r(X), linewidth=3, label="r(x)")

    # perform n_iters iterations of Newton's method
    plt.plot(x_bar, r(x_bar), 'o', label="Initial guess")
    for i in range(n_iters):
        # at the last iteration plot the Taylor expansion of r(x) around x_bar
        if(i==n_iters-1):
            plt.plot(X, r(x_bar) + grad_r(x_bar)*(X-x_bar), alpha=0.5, linewidth=2, label="Taylor expansion")
            
        # compute Newton's step
        delta_x = - r(x_bar) / grad_r(x_bar)
        x_bar += delta_x
        plt.plot(x_bar, r(x_bar), 'o', label="Iteration "+str(i+1))
        if(i==n_iters-1):
            plt.axvline(x=x_bar, ymin=1.0/23, ymax=(r(x_bar)+1)/23, ls=':', color='r', linewidth=2, alpha=0.5)

    print("Final value of r(x)=", r(x_bar))
    plt.xlabel("x")
    plt.ylim([-1, 22])
    plt.axhline(y=0, color='k')
    plt.axvline(x=0, color='k')
    plt.legend()
    plt.grid(True)
    plt.show()

In [8]:
slider = IntSlider(value=1, min=1, max=7, step=1, description='num iters')
interact(newtons_method_root_finding, n_iters=slider);

interactive(children=(IntSlider(value=1, description='num iters', max=7, min=1), Output()), _dom_classes=('wid…

# Newton's method for solving minimization problems

Let us now suppose that the system of equations that we want to solve with Newton's method is:
$$
\nabla f(x) = 0
$$

Newton's step then takes the form:
$$
\Delta x = - \nabla^2 f(\bar{x})^{-1} \, \nabla f(\bar{x})
$$
Contrary to gradient descent, Newton's step exploits the Hessian of the function $f(x)$.

#### Alternative Interpretation
Another way to interpret Newton's method is that at each step it minimizes a local 2-nd order approximation of $f(x)$:
$$ \min_{\Delta x} \, f(\bar{x}) + \nabla f(\bar{x})^T \, \Delta x + \frac{1}{2} \Delta x^T \, \nabla^2 f(\bar{x}) \, \Delta x $$
which results in the same Newton's step.

#### Example
Let's try to minimize the same ill-conditioned function that was leading to slow convergence with gradient descent.
$$ f(x, y) = 100x^2 + y^2 $$

In [8]:
from numpy.linalg import inv, norm

# Solve the minimization problem: min f(x)
def f_ill(x):
    return 100*x[0]**2 + x[1]**2

def f_ill_plot(x, y):
    return f_ill([x,y])

# function computing the gradient of f(x)
def grad_f_ill(x):
    return np.array([200*x[0], 2*x[1]])

# function computing the Hessian of f(x)
def hess_f_ill(x):
    return np.array([[200, 0],
                     [0, 2]])

In [23]:
def newtons_method_nd(n_iters, x_bar):
    # perform n_iters iterations of Newton's method
    history_x = [x_bar.copy()]
    history_f = [f_ill(x_bar)]
    for i in range(n_iters):
        # compute Newton's step
        delta_x = - inv(hess_f_ill(x_bar)) @ grad_f_ill(x_bar)
        x_bar += delta_x
        # store the updated value of x_bar and f(x_bar)
        history_x.append(x_bar.copy())
        history_f.append(f_ill(x_bar))

    print("Final value of x =  ", x_bar)
    print("Final value of f(x)=", f_ill(x_bar))
    print("Final value of the gradient of f(x)=", norm(grad_f_ill(x_bar)))

    # plot the function f(x)
    X, Y = np.meshgrid(np.linspace(-2, 2, 400), np.linspace(-2, 2, 400))
    Z = f_ill_plot(X, Y)
    fig = plt.figure(figsize=(12,5))
    ax = fig.subplots(1,2)
    ax[0].contour(X, Y, Z, levels=np.logspace(-1, 3, 20))
    history_x = np.array(history_x)
    ax[0].plot(history_x[:,0], history_x[:,1], 'ro-', label='Trajectory', alpha=0.5)
    ax[0].set_xlabel("x[0]"), ax[0].set_ylabel("x[1]")
    ax[0].legend()
    ax[0].grid(True)

    ax[1].plot(history_f)
    ax[1].set_ylabel("Cost function")
    ax[1].set_xlabel("Iteration")
    ax[1].grid(True)
    plt.show()

In [24]:
def newtons_method_0(n_iters):
    return newtons_method_nd(n_iters, x_bar=np.array([1.5, 1.5]))
interact(newtons_method_0, n_iters=IntSlider(value=1, min=1, max=5, step=1, description='num iters'));

interactive(children=(IntSlider(value=1, description='num iters', max=5, min=1), Output()), _dom_classes=('wid…

#### Another example
Let's try to minimize another function:
$$f(x) = x^4 - x$$

In [25]:
# Solve the minimization problem: min f(x)
def f(x):
    return x**4 -x

# function computing the gradient of f(x)
def grad_f(x):
    return 4*(x**3) - 1

# function computing the Hessian of f(x)
def hess_f(x):
    return 12*(x**2)

In [27]:
def newtons_method_1d(n_iters, x_bar, ylim):
    # plot the function f(x)
    X = np.linspace(-2.1, 3.5, 100)
    fig = plt.figure(figsize=(14,6))
    plt.plot(X, f(X), linewidth=3, label="f(x)")

    # perform n_iters iterations of Newton's method
    plt.plot(x_bar, f(x_bar), 'o', label="Initial guess")
    for i in range(n_iters):
        if(i==n_iters-1):
            # plot 2-nd order Taylor expansion of f(x) around x_bar
            plt.plot(X, f(x_bar) + grad_f(x_bar)*(X-x_bar) + 0.5*hess_f(x_bar)*((X-x_bar)**2), alpha=0.5, linewidth=2)
        # compute Newton's step
        delta_x = - grad_f(x_bar) / hess_f(x_bar)
        x_bar += delta_x
        # plot the updated value of x_bar
        plt.plot(x_bar, f(x_bar), 'o', label="Iteration "+str(i+1))
    plt.axvline(x=x_bar, ls=':', color='r', linewidth=2, alpha=0.5)

    print("Final value of x =  ", x_bar)
    print("Final value of f(x)=", f(x_bar))
    print("Final value of the gradient of f(x)=", grad_f(x_bar))
    plt.xlabel("x")
    plt.ylim(ylim)
    plt.axhline(y=0, color='k')
    plt.axvline(x=0, color='k')
    plt.legend()
    plt.grid(True)
    plt.show()

In [29]:
def newtons_method_1(n_iters):
    return newtons_method_1d(n_iters, x_bar=-2.0, ylim=[-4, 22])
interact(newtons_method_1, n_iters=IntSlider(value=1, min=1, max=12, step=1, description='num iters'));

interactive(children=(IntSlider(value=1, description='num iters', max=12, min=1), Output()), _dom_classes=('wi…

# Limitations of Vanilla Newton's Method
* In the example above, at the 5-th iteration the value of $f(x)$ increased from -0.16 to 91.
* In general Newton's method cannot guarantee monotonic improvement, nor convergence.
* For example trying to minimize $f(x) = \frac{1}{4} x^4 - x^2 + 2x$, starting from $\bar{x}\approx1$ the method oscillates forever.
    * $\nabla f(x) = x^3 − 2x + 2$
    * $\nabla^2 f(x) = 3 x^2 - 2$

In [30]:
# Solve the minimization problem: min f(x)
def f(x):
    return 0.25*(x**4) - x**2 + 2*x

# function computing the gradient of f(x)
def grad_f(x):
    return x**3 - 2*x + 2

# function computing the Hessian of f(x)
def hess_f(x):
    return 3* (x**2) - 2

In [31]:
def newtons_method_2(n_iters):
    return newtons_method_1d(n_iters, x_bar=0.99, ylim=[-5,3])
    
interact(newtons_method_2, n_iters=IntSlider(value=1, min=1, max=10, step=1, description='num iters'));

interactive(children=(IntSlider(value=1, description='num iters', max=10, min=1), Output()), _dom_classes=('wi…

# Introducing the line search
* To make Newton's method more robust we can introduce a **line search**
* Instead of taking a full Newton's step we can take a fraction $\alpha \in [0, 1]$ of it:
    * $x^{(i+1)} := x^{(i)} + \alpha \Delta x$
* Different ways to choose $\alpha$
* At the very least we want to ensure monotonic improvement: $f(x^{(i+1)}) < f(x^{(i)})$
* Even better, we may require a minimum improvement (Armijo's condition):
    * $f(x^{(i+1)}) \le f(x^{(i)}) - \gamma \, \alpha \, \nabla f^T \, \Delta x$
    * where $\gamma \in [0,1]$ is a user-defined hyper-parameter

## Backtracking Line Search
* Start with $\alpha=1$
* Compute Newton's step $\Delta x$
* Compute new value of $x$
* As long as the chosen decrease condition is not satisfied:
    * reduce $\alpha$
    * Compute new value of $x$

In [32]:
def newtons_method_w_line_search(n_iters, x_bar, ylim):
    # plot the function f(x)
    X = np.linspace(-2.1, 3.5, 100)
    fig = plt.figure(figsize=(12,5))
    plt.plot(X, f(X), linewidth=3, label="f(x)")

    # perform n_iters iterations of Newton's method
    plt.plot(x_bar, f(x_bar), 'o', label="Initial guess")
    for i in range(n_iters):
        if(i==n_iters-1):
            # plot 2-nd order Taylor expansion of f(x) around x_bar
            plt.plot(X, f(x_bar) + grad_f(x_bar)*(X-x_bar) + 0.5*hess_f(x_bar)*((X-x_bar)**2), alpha=0.5, linewidth=2)
        # compute Newton's step
        delta_x = - grad_f(x_bar) / hess_f(x_bar)
        # implement line search
        alpha = 1.0
        x_try = x_bar + alpha*delta_x
        while( f(x_try)>= f(x_bar)):
            # reduce alpha
            alpha *= 0.5
            # compute new value of x
            x_try = x_bar + alpha*delta_x
            # stop line search if alpha is too small
            if(alpha<1e-4):
                break
        x_bar = x_try
        
        # plot the updated value of x_bar
        plt.plot(x_bar, f(x_bar), 'o', label="Iteration "+str(i+1))
    plt.axvline(x=x_bar, ls=':', color='r', linewidth=2, alpha=0.5)

    print("Final value of x =  ", x_bar)
    print("Final value of f(x)=", f(x_bar))
    print("Final value of the gradient of f(x)=", grad_f(x_bar))
    plt.xlabel("x")
    plt.ylim(ylim)
    plt.axhline(y=0, color='k')
    plt.axvline(x=0, color='k')
    plt.legend()
    plt.grid(True)
    plt.show()

In [33]:
def newtons_method_3(n_iters):
    return newtons_method_w_line_search(n_iters, x_bar=1.0, ylim=[-4, 4])
    
interact(newtons_method_3, n_iters=IntSlider(value=1, min=1, max=10, step=1, description='num iters'));

interactive(children=(IntSlider(value=1, description='num iters', max=10, min=1), Output()), _dom_classes=('wi…

# Descent direction
* After the first iteration the search gets stuck around zero
* The quadratic approximation of $f(x)$ around zero has negative curvature: $\nabla^2 f(x) < 0$
* This implies that Netwon's step $\Delta x = -\nabla^2 f(x)^{-1} \, \nabla f(x)$ points in the same direction as $\nabla f(x)$
* In other words, $\Delta x$ is an **ascent** direction, whereas we want a **descent** direction
* How can we fix this?

# Hessian regularization
* To ensure that Newton's step gives us a **descent** direction we can regularize the Hessian $H$ so that $H>0$
 $$H = \nabla^2 f(x) + \lambda \, I$$
* where $\lambda>0$ is a regularization parameter that should be sufficiently large to ensure $H > 0$
    * We could find $\lambda$ by computing the minimum eigenvalue of $\nabla^2 f(x)$, call it $e_{min}$, and then setting $\lambda = \epsilon + \max(-e_{min}, 0)$, with $\epsilon>0$ being a small positive value
* Newton's step then becomes:
  $$ \Delta x = -H^{-1} \, \nabla f(x)$$
* If $H>0$ then $\Delta x$ is guaranteed to be a descent direction because:
  $$ \nabla f(x)^T \, \Delta x = - \nabla f(x)^T \, H^{-1} \, \nabla f(x) < 0 \qquad \forall x$$

In [34]:
def newtons_method_w_line_search_and_regul(n_iters, eps, x_bar, ylim):
    # plot the function f(x)
    X = np.linspace(-3., 3., 100)
    fig = plt.figure(figsize=(12,5))
    plt.plot(X, f(X), linewidth=3, label="f(x)")

    # perform n_iters iterations of Newton's method
    plt.plot(x_bar, f(x_bar), 'o', label="Initial guess")
    for i in range(n_iters):
        H = hess_f(x_bar) # compute Hessian
        # since the Hessian is a scalar it is equal to its unique eigenvalue
        lmbda = eps + max(-H, 0)
        H += lmbda # regularize the Hessian
        # at the last iteration plot 2-nd order Taylor expansion of f(x) around x_bar
        if(i==n_iters-1):
            plt.plot(X, f(x_bar) + grad_f(x_bar)*(X-x_bar) + 0.5*H*((X-x_bar)**2), alpha=0.5, linewidth=2)
        delta_x = - grad_f(x_bar) / H # compute Newton's step
        # implement line search
        alpha = 1.0
        x_try = x_bar + alpha*delta_x
        while( f(x_try)>= f(x_bar)):
            alpha *= 0.5 # reduce alpha
            # compute new value of x
            x_try = x_bar + alpha*delta_x
            # stop line search if alpha is too small to avoid looping forever
            if(alpha<1e-6):
                break
        print(f"Iter {i}, H={H:.3f}, alpha={alpha}")
        x_bar = x_try
        
        # plot the updated value of x_bar
        plt.plot(x_bar, f(x_bar), 'o', label="Iteration "+str(i+1))
    plt.axvline(x=x_bar, ls=':', color='r', linewidth=2, alpha=0.5)

    print("Final value of x =  ", x_bar)
    print("Final value of f(x)=", f(x_bar))
    print("Final value of the gradient of f(x)=", grad_f(x_bar))
    plt.xlabel("x")
    plt.ylim(ylim)
    plt.axhline(y=0, color='k')
    plt.axvline(x=0, color='k')
    plt.legend()
    plt.grid(True)
    plt.show()

In [35]:
def newtons_method_4(n_iters):
    return newtons_method_w_line_search_and_regul(n_iters, eps=1e-4, x_bar=1.0, ylim=[-5, 4])
    
interact(newtons_method_4, n_iters=IntSlider(value=1, min=1, max=10, step=1, description='num iters'));

interactive(children=(IntSlider(value=1, description='num iters', max=10, min=1), Output()), _dom_classes=('wi…

# Summary
* Newton's method exploits second-order derivatives to ensure **faster convergence** than gradient descent on ill-conditioned functions
* **Line search** is introduced to ensure monotonic improvement at each iteration
* **Hessian regularization** is introduced to ensure improvement is possible even with negative-definite Hessian
* These ideas can be extended to **constrained** optimization problems, which are subject to equality and inequality constraints